In [10]:
import os, sys, re
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import networkx as nx

from tqdm import tqdm
from pyvis.network import Network
from persiantools.jdatetime import JalaliDate

In [11]:
JalaliDate.today()

JalaliDate(1403, 1, 19, Yekshanbeh)

In [12]:
provinces = ['Gilan', 'Golestan', 'Mazandaran']

# Read city-translation.csv
city_translation = pd.read_csv("./helpers/city-translation.csv")
road_translation = pd.read_csv("./helpers/road-translation.csv")

In [13]:
# Unzip the data
os.system("unzip data.zip")

Archive:  data.zip
   creating: data/
   creating: data/Golestan/
  inflating: data/Golestan/کردک�?�? - ب�?در ترک�?�?.csv  
  inflating: data/Golestan/گ�?بد - ا�?�? �?�?ا.csv  
  inflating: data/Golestan/�?�?�?�?دشت - گ�?بد.csv  
  inflating: data/Golestan/آزادش�?ر - خ�?ش �?�?�?ا�? (آزادش�?ر - شا�?ر�?د).csv  
  inflating: data/Golestan/س�? را�?�? ک�?ا�?�? - ت�?گرا�?.csv  
  inflating: data/Golestan/کردک�?�? - گرگا�? (�?د�?�?).csv  
  inflating: data/Golestan/گا�?�?کش - �?�?�?�?دشت.csv  
  inflating: data/Golestan/ب�?در ترک�?�? - س�? را�?�? �?سا�?�?.csv  
  inflating: data/Golestan/آزادش�?ر - گ�?بد.csv  
  inflating: data/Golestan/آ�? �?�?ا - گ�?بد.csv  
  inflating: data/Golestan/آزادش�?ر - �?�?�?�?دشت.csv  
  inflating: data/Golestan/گرگا�? - ع�?�? ا�?باد.csv  
  inflating: data/Golestan/گ�?بد - �?�?�?�?دشت.csv  
  inflating: data/Golestan/کردک�?�? - گ�?�?گا�?.csv  
  inflating: data/Golestan/آ�? �?�?ا - ب�?در ترک�?�?.csv  
  inflating: data/Golestan/ع�?�? ا�?باد - گرگا�?.csv  
  infl

0

In [14]:
# Create a multi-directed graph
G = nx.MultiDiGraph()

# Get starting and ending interval
start_date = input("Enter the start date in format (yyyy/mm/dd): ")
end_date = input("Enter the end date in format (yyyy/mm/dd): ")

# Extract year, month, and day
start_date = JalaliDate(
    int(start_date.split('/')[0]),
    int(start_date.split('/')[1]),
    int(start_date.split('/')[2])
)

end_date = JalaliDate(
    int(end_date.split('/')[0]),
    int(end_date.split('/')[1]),
    int(end_date.split('/')[2])
)

classes = input("Enter the classes you want to see separated by a comma (e.g. 1,2,3): ")
classes = classes.split(',')
classes = [int(c) for c in classes]

for province in provinces:
    files = os.listdir(f'./data/{province}')
    for file in files:
        name = re.sub(r'.csv', '', file)

        nodes = name
        edge = ''
        if '(' in name:
            name = name.split('(')
            nodes = name[0].strip()
            edge = name[1].strip()
            edge = edge[:-1]
        
        starting_node = nodes.split('-')[0].strip()
        ending_node = nodes.split('-')[1].strip()

        # English names
        if city_translation[city_translation['Persian'] == starting_node].shape[0] > 0:
            starting_node = city_translation[city_translation['Persian'] == starting_node]['English'].values[0]
        else:
            print(f"Starting node {starting_node} not found in city_translation.")
        if city_translation[city_translation['Persian'] == ending_node].shape[0] > 0:
            ending_node = city_translation[city_translation['Persian'] == ending_node]['English'].values[0]
        else:
            print(f"Ending node {ending_node} not found in city_translation.")

        df = pd.read_csv(f'./data/{province}/{file}')

        df['Start Date'] = df['Start Time'].apply(lambda x: JalaliDate(
        int(x.split(' ')[0].split('/')[0]),
        int(x.split(' ')[0].split('/')[1]),
        int(x.split(' ')[0].split('/')[2])
        ))

        df['End Date'] = df['End Time'].apply(lambda x: JalaliDate(
        int(x.split(' ')[0].split('/')[0]),
        int(x.split(' ')[0].split('/')[1]),
        int(x.split(' ')[0].split('/')[2])
        ))

        weight = 0
        # For all rows in df where the start date is between the start and end date
        for index, row in df.iterrows():
            if row['Start Date'] >= start_date and row['Start Date'] < end_date:
                for c in classes:
                    weight += row[f'Number of Class {c} Vehicles']
        

        if 'قا' in starting_node:
            starting_node = 'Qaemshahr'
        if 'قا' in ending_node:
            ending_node = 'Qaemshahr'     

        if starting_node not in G:
            G.add_node(starting_node)
        if ending_node not in G:
            G.add_node(ending_node)

        if road_translation[road_translation['Persian'] == edge].shape[0] > 0:
            edge = road_translation[road_translation['Persian'] == edge]['English'].values[0]
        G.add_edge(starting_node, ending_node, label=f'{edge}({weight})', weight=np.log10(weight + 1))

nt = Network('1000px', '1500px', directed=True, select_menu=True, filter_menu=True, notebook=False)
nt.from_nx(G)

for node in nt.nodes:
    incoming_edges = G.in_edges(node['id'])
    outgoing_edges = G.out_edges(node['id'])
    net = 0
    for edge in incoming_edges:
        # For each value
        for key, value in G[edge[0]][edge[1]].items():
            # Extract what's in value['label'] between the parentheses
            value = value['label']
            value = value[value.find('(')+1:value.find(')')]
            net += int(value)
    for edge in outgoing_edges:
        for key, value in G[edge[0]][edge[1]].items():
            value = value['label']
            value = value[value.find('(')+1:value.find(')')]
            net -= int(value)
    if net > 0:
        # Set size of node to be proportional to net
        node['size'] = 5 * np.log10(net + 10)
        node['color'] = 'green'
    elif net < 0:
        node['size'] = 5 * np.log10(-net + 10)
        node['color'] = 'red'
    else:
        node['size'] = 5
        node['color'] = 'blue'
    node['label'] = node['id'] + f'({net})'

nt.toggle_physics(True)
nt.force_atlas_2based(spring_length=150, central_gravity=0.003, gravity=-25, damping=0.9, overlap=1)
nt.show('map.html', notebook=False)
os.system('open map.html')

map.html


0